In [ ]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 1.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Открываем файл Excel
file1 = pd.ExcelFile('/content/Rating_lists_date.xlsx')

# Загружаем данные из листа в DataFrame
df = file1.parse()

# Удаляем 4, 7 и 13 столбцы
columns_to_delete = [3, 6, 12]
df = df.drop(df.columns[columns_to_delete], axis=1)

# Словарь кодовых слов и соответствующих листов
code_words = {
    'Автоматизация и робототехнические системы /направления 15.03.04 и 15.03.06/ бакалавриат, бюджет, очная форма (Бюджет, Очная)': 'Меха + АТПП',
    'Электроника и автоматика физических установок специалитет, бюджет, очная форма (Бюджет, Очная)': 'ЭиА',
    'Приборостроение бакалавриат, бюджет, очная форма (Бюджет, Очная)': 'Приборка',
    'Ядерная физика и атомная энергетика /направления 14.03.01 и 14.03.02/, бакалавриат, бюджет, очная форма (Бюджет, Очная)': 'ЯФиТ',
    'Системный анализ и управление бакалавриат,бюджет, очная форма (Бюджет, Очная)': 'СА'
}

# Создаем новый Excel-файл
writer = pd.ExcelWriter('/content/Debugging_file.xlsx', engine='openpyxl')

# Проходимся по каждому кодовому слову
for keyword, sheet_name in code_words.items():
    start_row = None
    end_row = None

    # Ищем кодовое слово в DataFrame и запоминаем диапазон строк
    for index, row in df.iterrows():
        if keyword in row.values:
            start_row = index
            break

    if start_row is not None:
        for index, row in df.iloc[start_row:].iterrows():
            if pd.isnull(row.values).all():
                end_row = index
                break

    # Создаем новый DataFrame с диапазоном строк
    new_df = df.iloc[start_row:end_row]

    # Сохраняем DataFrame на соответствующем листе
    new_df.to_excel(writer, sheet_name=sheet_name, index=False)

# Сохраняем и закрываем файл
writer.save()

#############################################################################################################################################################################3

# Загрузка данных из файлов
file1 = pd.ExcelFile('/content/Main_file.xlsx')
file2 = pd.ExcelFile('/content/Debugging_file.xlsx')

# Получение списка названий листов
sheet_names = file1.sheet_names

# Создание объекта pd.ExcelWriter для записи изменений в новый файл
writer = pd.ExcelWriter('/content/New_Main_file.xlsx', engine='xlsxwriter')

# Итерация по листам
for sheet_name in sheet_names:
    # Загрузка данных для текущего листа из обоих файлов
    df1 = file1.parse(sheet_name)
    df2 = file2.parse(sheet_name)

    # Создание пустого списка для хранения добавленных данных
    additional_data = []

    # Перебор строк во втором файле
    for index2, row2 in df2.iterrows():
        number_id_2 = row2[2]  # Индекс третьего столбца (нумерация начинается с 0)

        # Перебор строк в первом файле
        for index1, row1 in df1.iterrows():
            number_id_1 = row1[2]  # Индекс третьего столбца (нумерация начинается с 0)

            # Проверка на совпадение номеров личных дел
            if number_id_2 == number_id_1:
                # Добавление данных из столбцов 9-17 из первого файла во второй файл
                additional_data = row1[10:18].values.tolist()
                df2.loc[index2, 'Телефон (смотреть по оргу)'] = additional_data[0]
                df2.loc[index2, 'Дата звонков (указывать все)'] = additional_data[1]
                df2.loc[index2, 'Кто звонил (не зачеркивать)'] = additional_data[2]
                df2.loc[index2, 'Кто постоянно общается'] = additional_data[3]
                df2.loc[index2, 'Где держишь связь (если держишь)'] = additional_data[4]
                df2.loc[index2, 'Поле под комментарии 1'] = additional_data[5]
                df2.loc[index2, 'Поле под комментарии 2'] = additional_data[6]
                df2.loc[index2, 'Первое приоритетное направление'] = additional_data[7]

    # Сохранение изменений в текущем листе
    df2.to_excel(writer, sheet_name=sheet_name, index=False)

    # Получение объекта workbook и worksheet
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # Установка автоширины для столбцов 4, 8, 9, 10, 11 и 12
    for col_num in [3, 7, 8, 9, 10, 11]:
        max_width = max(
            df2.iloc[:, col_num].astype(str).map(len).max(),
            len(df2.columns[col_num])
        )
        worksheet.set_column(col_num, col_num, max_width)

# Завершение записи и сохранение изменений в новом файле
writer.save()

<ipython-input-3-b0499b4ca13b>:49: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [ ]:
# Открываем файл Excel
file1 = pd.ExcelFile('/Rating_lists_date.xlsx')

# Загружаем данные из листа в DataFrame
df = file1.parse()

# Удаляем строки, где в 6 столбце не стоит "1" или в 10 столбце не стоит "Оригинал"
df = df[(df.iloc[:, 5] == 1) & (df.iloc[:, 9] == "Оригинал")]

# Удаляем 1, 2, 4, 6, 7, 10 и 13 столбцы
columns_to_delete = [0, 1, 3, 5, 6, 9, 12]
df = df.drop(df.columns[columns_to_delete], axis=1)

# Удаляем строки, где в 1 столбце нет подстроки "НИЯУ МИФИ"
df = df[df.iloc[:, 0].str.contains("НИЯУ МИФИ", na=False)]

# Сортируем данные по 4 столбцу по убыванию
df = df.sort_values(by=df.columns[3], ascending=False)

# Сравниваем две соседние строки по первому столбцу и удаляем нижнюю, если они совпадают
df = df[~df.iloc[:, 0].duplicated(keep='last')]

# Создаем новый Excel-файл
writer = pd.ExcelWriter('/content/only_first_priorities.xlsx', engine='openpyxl')

# Записываем данные в новый файл
df.to_excel(writer, index=False)

# Завершение записи и сохранение изменений в новом файле
writer.save()

<ipython-input-12-c3b666d4ccde>:32: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
